In [2]:
import pandas as pd

df = pd.read_csv("./swissvotes-dataset.csv", delimiter=";", na_values=".")
# Exclude data from votes with missing values (very first and of the last)
df = df.iloc[1:-12, :]
# Extract some data rows for comparison and verification
volk = df["volk"].iloc[:]
stand = df["stand"].iloc[:]
kt_ja = df["kt-ja"].iloc[:]
kt_nein = df["kt-nein"].iloc[:]
ktjaproz = df["ktjaproz"].iloc[:]

staende = kt_ja + kt_nein
invalid_results = (staende > 23).sum() + (staende < 22).sum()
print("Number of invalid staende results: {}".format(invalid_results))

Number of invalid staende results: 0


In [3]:
#(df["volk"] == df["volkja-proz"] > 50.0).sum()
volk_wrong = (df["volk"] != (df["volkja-proz"] > 50.0)).sum()
print("Check calculation of volk: {}".format(volk_wrong))
delta_ktjaproz = (df["ktjaproz"] - (100.0 * df["kt-ja"] / staende)).abs().sum()
print("Total absolute sum of deltas for ktjaproz: {}".format(delta_ktjaproz))

delta_volkja_proz = (df["volkja-proz"] - (100.0 * df["volkja"] / df["gultig"])).abs().sum()
print("Total absolute sum of deltas for volkja-proz: {}".format(delta_volkja_proz))

delta_bet = (df["bet"] - (100.0 * df["stimmen"] / df["berecht"])).abs().sum()
print("Total absolute sum of deltas for bet: {}".format(delta_bet))

invalid_berecht = (df["berecht"] < df["stimmen"]).sum()
print("Number of invalid berecht: {}".format(invalid_berecht))

delta_stimmen = (df["stimmen"] - df["leer"] - df["ungultig"] - df["gultig"]).abs().sum()
print("Total absolute sum of deltas for stimmen: {}".format(delta_stimmen))

delta_gultig = (df["gultig"] - df["volkja"] - df["volknein"]).abs().sum()
print("Total absolute sum of deltas for gultig: {}".format(delta_gultig))
(df["gultig"] - df["volkja"] - df["volknein"] > 0).sum()

Check calculation of volk: 0
Total absolute sum of deltas for ktjaproz: 1.058142292489796
Total absolute sum of deltas for volkja-proz: 1.5598724290052153
Total absolute sum of deltas for bet: 1.53731533820525
Number of invalid berecht: 0
Total absolute sum of deltas for stimmen: 55086.0
Total absolute sum of deltas for gultig: 1638038.0


27